# Demo 1: netUnicorn basic usage

In this demo we will show the basic usage of netUnicorn platform, including pipeline definition, experiment creation, compilation and deployment, and also result analysis.

We will need to import some abstractions and prepared tasks for this demo:

In [ ]:
# Remote client to connect to remote netunicorn instance
from netunicorn.client.remote import RemoteClient

# Basic abstractions - pipeline and experiment
from netunicorn.base import Pipeline, Experiment

# And different prepared tasks:
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest           # Ookla SpeedTest
from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture   # Start and stop tcpdump
from netunicorn.library.tasks.upload.fileio import UploadToFileIO                     # Upload files to an online storage

We imported all the tasks that we will use and now need to create a desired pipeline for the data collection intent.

We create a Pipeline object and use `.then` operation to create a new stage: all tasks on this stage would be executed only after the previous stage finished.

In [ ]:
pipeline = (
    Pipeline()
    .then(StartCapture(filepath="/tmp/capture.pcap", name="capture"))
    .then(SpeedTest(name="speedtest"))
    .then(StopNamedCapture(start_capture_task_name="capture", name="stop_capture"))
    .then(UploadToFileIO(filepath="/tmp/capture.pcap", expires="1d", name="upload_data"))
)

Now let's connect to our local netUnicorn instance and get nodes that we will use for this experiment.

In [ ]:
client = RemoteClient(endpoint='http://localhost:26611', login='test', password='test')
print(f"netunicorn instance is healthy: {client.healthcheck()}")

nodes = client.get_nodes()
print(f"Nodes: {nodes}")

As you can see, currently we have only one node - local docker host (for purposes of the tutorial). We can deploy any number of pipelines to this host to work in parallel, but now let's just create a simple Experiment with a single pipeline that we defined.

In [ ]:
experiment = Experiment().map(pipeline, nodes.take(1))
print(experiment)

Currently experiment is defined, but not yet prepared. For this, we need to send it to the netUnicorn instance and ask to prepare it.

In [ ]:
client.delete_experiment(experiment_label)

In [ ]:
experiment_label = "session2_demo1_example"
client.prepare_experiment(experiment, experiment_label)

Preparation consists of compiling a Docker image with all dependencies and code inside and distributing it to all nodes participating in the experiment. See the presentation for additional details.

Let's check the experiment status and wait until it's "READY" for execution.

In [ ]:
info = client.get_experiment_status(experiment_label)
info

When preparation is finished, we can verify that all nodes are ready and start the data collection experiment.

In [ ]:
client.start_execution(experiment_label)

Starting the experiment sends a command to all participating nodes to actually start the required scripts or container to execute, and collect results afterward.

As with preparation, let's wait until the experiment is "FINISHED".

In [ ]:
info = client.get_experiment_status(experiment_label)
info.status

After experiment is finished and we received the full experiment status, we can explore it - see nodes participating in the experiment, their execution results, and execution result of each task in the pipeline.

In [ ]:
from returns.pipeline import is_successful

for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    result, log = report.result  # report stores results of execution and corresponding log
    
    # result is a returns.result.Result object, could be Success of Failure
    print(f"Result is: {type(result)}\n")
    data = result.unwrap() if is_successful(result) else result
    for key, value in data.items():
        print(f"{key}: {value}")

    print()
    # we also can explore logs
    for line in log:
        print(line.strip())
    print()